In [1]:
## Data

numLines = 2
numTypeGlue = 3

# max number of workers per week
maxW = 20

# weekly pay per worker in production line l 
p = {1:500 , 2:900 }

#setup costs for each production line
s = {1:5000, 2:4000}

## weekly production per worker for each type of glue in each production line 

a = {(1,1):20 , (2,1):30 ,(3,1):40 , 
    (1,2):50 , (2,2):35, (3,2):45, 
    }

## minimum number of units of glue to produce per week
b = {1:800 , 2:750 , 3:100}




In [2]:
import pyomo.environ as pe
import pyomo.opt as po

In [3]:
model = pe.ConcreteModel()
model.G = pe.RangeSet(1,numTypeGlue)
model.L = pe.RangeSet(1,numLines)

In [4]:
## Variables
model.x = pe.Var(model.L, domain = pe.NonNegativeIntegers)
model.y = pe.Var(model.L, domain = pe.Binary)

In [5]:
## Objective function
objExpr = sum(model.x[l] * p[l] + model.y[l] * s[l] for l in model.L)
model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [6]:
## Constraints

## Min production for each type of glue
model.minProdGlue = pe.ConstraintList()
for g in model.G:
    expression = sum(a[g,l] * model.x[l] for l in model.L) >= b[g]
    model.minProdGlue.add(expression)

## bigM constraints to take care of setup costs and max number of workers
model.bigMconstr = pe.ConstraintList()
for l in model.L:
    expression = model.x[l] <= maxW * model.y[l]
    model.bigMconstr.add(expression)


In [7]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee= True)


Set parameter Username
Academic license - for non-commercial use only - expires 2022-12-10
Read LP format model from file C:\Users\rpo330\AppData\Local\Temp\tmpcunfpmyp.pyomo.lp
Reading time = 0.00 seconds
x5: 6 rows, 5 columns, 11 nonzeros
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6 rows, 5 columns and 11 nonzeros
Model fingerprint: 0x036b6b31
Variable types: 1 continuous, 4 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [5e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]
Presolve removed 4 rows and 3 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)
Found heuristic solution: objective 27600.000000
Found heuristic solution: objective 26200.000000

Root relaxation: objective 2.506250e+04, 2 iterations, 0.00 seconds (0.00 work 

In [8]:
print(result.solver.status)
print(result.solver.termination_condition)

ok
optimal


In [9]:
print("Total cost = "+ str(pe.value(model.obj)))
print("Optimal solution:")
for l in model.L: 
        if(pe.value(model.x[l]) > 0):
            print(str(model.x[l])+" = "+str(pe.value(model.x[l])))
        if(pe.value(model.y[l]) > 0):
             print(str(model.y[l])+" = "+str(pe.value(model.y[l])))

Total cost = 25300.0
Optimal solution:
x[1] = 11.0
y[1] = 1.0
x[2] = 12.0
y[2] = 1.0
